In [1]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph,
                         DiagramBuilder, JointSliders, LeafSystem,
                         MeshcatVisualizer, Parser, RigidTransform,
                         RollPitchYaw, StartMeshcat)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


In [14]:
class PrintPose(LeafSystem):
    
    def __init__(self, body_index): 
        LeafSystem.__init__(self)
        self._body_index = body_index
        # abstract input aka idk the input shape (matrix, tensor, scalar)
        self.DeclareAbstractInputPort(
            'body_pose',
            AbstractValue.Make([RigidTransform()])
        )
        
        #setting a publish event on a schedule
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context): 
        pose = self.get_input_port().Eval(context)[self._body_index]
        
        # print part
        print(pose)
        print('gripper pose at: ' +np.array2string(
            pose.translation(), formatter={'float': lambda x: "{:3.2f}".format(x)}
        ))
        print('gripper roll-pitch-yaw (Rad): ' + np.array2string(
            RollPitchYaw(pose.rotation()).vector(), 
            formatter={'float': lambda x: "{:3.2f}".format(x)}
        ))
        
        clear_output(wait=True) # print just the last result
        



In [ ]:
def gripper_forward_kinematics_exe(): 
    
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser) # allow to search for extra pkg in given dir
    parser.AddAllModelsFromFile(FindResource("models/iiwa_and_wsg.dmd.yaml"))
    plant.Finalize()
    
    #draw frames: 
    
    BODY_NAMES =  [f'iiwa_link_{i}' for i in range(1,7)]
    BODY_NAMES.append('body')
    
    for body_name in BODY_NAMES: 
        AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)
    
    
    # visualizer
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )
    
    
    # CONNECT PRINT POSE SYSTEM 
    wsg = plant.GetModelInstanceByName("wsg")
    gripper = plant.GetBodyByName('body', wsg)
    print_pose = builder.AddSystem(PrintPose(gripper.index()))
    builder.Connect(
        plant.get_body_poses_output_port(), 
        print_pose.get_input_port()
    )
    
    # gui for meshcat : jointSlider allow me to controll the joint manually
    default_interactive_timeout = None
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()
    
    
gripper_forward_kinematics_exe()

RigidTransform(
  R=RotationMatrix([
    [6.123377970758711e-17, -6.123189642323984e-17, 1.0000000000000004],
    [1.0000000000000004, -9.808978045275118e-17, -6.123220754656784e-17],
    [-9.808982757910806e-17, 1.0000000000000007, 6.12323514879913e-17],
  ]),
  p=[3.295969108337938e-22, 1.425863531256601e-17, 1.31],
)
gripper pose at: [0.00 0.00 1.31]
gripper roll-pitch-yaw (Rad): [1.57 0.00 1.57]
